# Criteo Click Through Prediction using ThirdAI's Universal Deep Transformer(UDT) APIs
This notebook shows how to build a Click Through Prediction model using ThirdAI's UDT.

In [34]:
!pip3 install thirdai --upgrade

Requirement already up-to-date: thirdai in /home/pratik/.local/lib/python3.8/site-packages (0.5.10)


# Download and Process Dataset

We will now create a sample dataset for training. Criteo-Click Logs dataset can be downloaded from https://ailab.criteo.com. Once datasets are downloaded convert the datasets into csv format. 

Column headers: `<label>` is titled as `<label>`, `<integer feature 1>` is titled as `numeric_1` and `<categorical feature 1>` as `cat_1`

<span style="color:red">Disclaimer: Remember that the following model is trained on sample data, to achieve better AUC train on the original criteo-dataset.</span>

In [48]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

TRAIN_FILE = 'train_file.txt'
TEST_FILE = 'test_file.txt'

header = 'label,numeric_1,numeric_2,numeric_3,numeric_4,numeric_5,numeric_6,numeric_7,numeric_8,numeric_9,numeric_10,numeric_11,numeric_12,numeric_13,'
header += 'cat_1,cat_2,cat_3,cat_4,cat_5,cat_6,cat_7,cat_8,cat_9,cat_10,cat_11,cat_12,cat_13,cat_14,cat_15,cat_16,cat_17,cat_18,cat_19,cat_20,cat_21,'
header += 'cat_22,cat_23,cat_24,cat_25,cat_26'

column_header=header.split(",")

numeric_data = np.random.randint(1500, size=(1200, 13))
categorical_data = np.random.randint(10000, size=(1200, 26))
label = np.random.randint(2, size=(1200, 1))

data_X , data_Y = np.concatenate((numeric_data, categorical_data), axis=1), label

train_data = np.concatenate((data_Y[:1000], data_X[:1000]), axis=1)
train_df = pd.DataFrame(train_data, columns=column_header)
train_df.to_csv(TRAIN_FILE, index=False)


test_data = np.concatenate((data_Y[1000:], data_X[1000:]), axis=1)
train_df = pd.DataFrame(test_data, columns=column_header)
train_df.to_csv(TEST_FILE, index=False)

train_df.head(10)

,label,numeric_1,numeric_2,numeric_3,numeric_4,numeric_5,numeric_6,numeric_7,numeric_8,numeric_9,...,cat_17,cat_18,cat_19,cat_20,cat_21,cat_22,cat_23,cat_24,cat_25,cat_26
0,1,821,282,714,794,1150,661,1122,251,803,...,3844,500,8872,4589,6448,979,5767,4618,9035,9755
1,1,1374,837,326,168,1322,634,901,255,543,...,9100,5061,2274,3996,7415,5935,3863,9794,1497,1969
2,0,783,913,816,129,679,631,750,170,555,...,3634,2176,6074,6546,9904,7893,2002,3040,2341,2647
3,1,81,951,1481,242,119,1416,1104,1194,631,...,9441,654,135,2814,373,2619,1189,6809,2172,2153
4,0,1194,659,1164,700,696,1432,1025,1405,603,...,3038,735,4117,969,4216,7534,9342,6738,6724,9090
5,0,250,305,97,938,283,689,1050,971,414,...,5362,3688,940,1855,9873,4375,3624,7320,7725,9557
6,1,1184,648,101,1249,380,756,711,191,1155,...,1687,5238,5865,4626,2419,9051,1169,4712,6446,5744
7,0,1216,1489,15,84,1493,501,1389,314,437,...,3585,9283,7452,629,2260,6913,7101,9258,1443,7464
8,0,1422,952,1228,227,1440,1350,18,417,1416,...,2297,613,8052,7527,1475,8788,1358,127,3851,3885
9,0,57,1181,811,70,630,999,790,1153,719,...,3620,7907,4499,1849,2538,4511,2920,9438,4102,1291


# UDT Initialization


We can create a UDT model specific for Criteo Click Through Prediction as follows. Here we define Bolt Datatype for all the columns, specify the target, n_target classes and the embedding dimension for UDT. 

In [49]:
from thirdai import bolt
import numpy as np
from sklearn.metrics import roc_auc_score


tabular_model = bolt.UniversalDeepTransformer(
    data_types={
        "numeric_1": bolt.types.numerical(range=(0, 1500)),
        "numeric_2": bolt.types.numerical(range=(0, 1500)),
        "numeric_3": bolt.types.numerical(range=(0, 1500)),
        "numeric_4": bolt.types.numerical(range=(0, 1500)),
        "numeric_5": bolt.types.numerical(range=(0, 1500)),
        "numeric_6": bolt.types.numerical(range=(0, 1500)),
        "numeric_7": bolt.types.numerical(range=(0, 1500)),
        "numeric_8": bolt.types.numerical(range=(0, 1500)),
        "numeric_9": bolt.types.numerical(range=(0, 1500)),
        "numeric_10": bolt.types.numerical(range=(0, 1500)),
        "numeric_11": bolt.types.numerical(range=(0, 1500)),
        "numeric_12": bolt.types.numerical(range=(0, 1500)),
        "numeric_13": bolt.types.numerical(range=(0, 1500)),
        "cat_1": bolt.types.categorical(),
        "cat_2": bolt.types.categorical(),
        "cat_3": bolt.types.categorical(),
        "cat_4": bolt.types.categorical(),
        "cat_5": bolt.types.categorical(),
        "cat_6": bolt.types.categorical(),
        "cat_7": bolt.types.categorical(),
        "cat_8": bolt.types.categorical(),
        "cat_9": bolt.types.categorical(),
        "cat_10": bolt.types.categorical(),
        "cat_11": bolt.types.categorical(),
        "cat_12": bolt.types.categorical(),
        "cat_13": bolt.types.categorical(),
        "cat_14": bolt.types.categorical(),
        "cat_15": bolt.types.categorical(),
        "cat_16": bolt.types.categorical(),
        "cat_17": bolt.types.categorical(),
        "cat_18": bolt.types.categorical(),
        "cat_19": bolt.types.categorical(),
        "cat_20": bolt.types.categorical(),
        "cat_21": bolt.types.categorical(),
        "cat_22": bolt.types.categorical(),
        "cat_23": bolt.types.categorical(),
        "cat_24": bolt.types.categorical(),
        "cat_25": bolt.types.categorical(),
        "cat_26": bolt.types.categorical(),
        "label": bolt.types.categorical(),
    },
    target="label",
    n_target_classes=2,
    options={"embedding_dimension": 512},
)

# Training

We will now train the UDT with just one line of code. Here we are specifying the training file name, number of epochs to train and maximum number of batches we want to load in memory at once. `max_in_memory_batches` support loaing data in a streaming fashion. You can change `max_in_memory_batches` based on your own memory specification.

In [50]:


tabular_model.train(filename=TRAIN_FILE, epochs=1, max_in_memory_batches=1000)



loading data | source 'train_file.txt' | vectors 1000 | batches 1 | time 0s | complete

train | epoch 0 | train_steps 1 | {} | train_batches 1 | time 0s | complete     



# Evaluation

Evaluating the UDT is also just one line of code. Evaluate function by default returns activations. Then, we are using sklearn's roc_auc_score to calculate roc_auc_score for the model we trained.

In [52]:
activations = tabular_model.evaluate(filename=TEST_FILE, metrics=["categorical_accuracy"])

true_labels = np.zeros(activations.shape[0], dtype=np.float32)
with open(TEST_FILE) as f:
    header = f.readline()
    count = 0
    for line in f:
        true_labels[count] = np.float32(line.split(",")[0])
        count += 1

roc_auc = roc_auc_score(true_labels, activations[:, 1])

print("ROC_AUC:", roc_auc)



loading data | source 'test_file.txt' | vectors 200 | batches 1 | time 0s | complete

evaluate | epoch 1 | train_steps 1 | {categorical_accuracy: 0.57} | eval_batches 1 | time 7ms

ROC_AUC: 0.5039386189258311


In [53]:
import os
os.remove(TRAIN_FILE)
os.remove(TEST_FILE)